In [36]:
pip install streamlit

In [37]:
pip install ta

In [38]:
pip install backtrader

In [104]:
import numpy as np
import pandas as pd
import yfinance as yf
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout, Conv1D, MaxPooling1D
from tensorflow.keras.layers import Input, Flatten, BatchNormalization, concatenate
from tensorflow.keras.regularizers import l2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from ta.volatility import BollingerBands
from ta.momentum import StochasticOscillator
import matplotlib.pyplot as plt
import backtrader as bt
import streamlit as st

In [105]:
# Загрузка данных из файла
def load_data_from_file(file_path):
  try:
    data = pd.read_csv(file_path)
    data['Date'] = pd.to_datetime(data['Date'])
    data.set_index('Date', inplace=True)
    return data
  except FileNotFoundError:
      print(f"Файл '{file_path}' не найден.")
      return None

# Загрузка данных из yaho finance
def load_data_from_yf(ticker, timeframe='1d'):
  df = yf.download(ticker, interval=timeframe)
  df = df.reset_index()
  if isinstance(df.columns, pd.MultiIndex):
      df.columns = df.columns.droplevel(level=1)
  df['Date'] = pd.to_datetime(df['Date'])
  df.set_index('Date', inplace=True)
  return df

In [106]:
# Индикатор RSI
def rsi(close_prices, n=14):
  delta = close_prices.diff().dropna()
  up = delta.clip(lower=0)
  down = -delta.clip(upper=0)
  rs = up.ewm(span=n, adjust=False).mean() / down.ewm(span=n, adjust=False).mean()
  return 100 - (100 / (1 + rs))

In [107]:
# Колонки для нормализации
columns_to_scale = ['Open', 'High', 'Low', 'Close', 'Volume', 'Mean', 'Value']
# Колонки, которые не нужно нормализовать
columns_not_to_scale = ['Predict', 'SMA', 'RSI', 'MACD', 'Signal_Line', 'ATR', 'bb_high', 'bb_low', 'so']

# Предобработка данных
def preprocess_data(data, window_size=30):
  # Добавление новых признаков
  data['Mean'] = (data['Open']+data['High']+data['Low']+data['Close']) / 4 # Средняя цена
  data['Value'] = data['Volume'] * data['Mean'] # Объём в деньгах
  data['SMA'] = data['Close'].rolling(window_size).mean()  # Скользящая средняя
  data['RSI'] = rsi(data['Close'], n=window_size)  # Индекс относительной силы
  data['MACD'] = data['Close'].ewm(span=12, adjust=False).mean() - data['Close'].ewm(span=26, adjust=False).mean()
  data['Signal_Line'] = data['MACD'].ewm(span=9, adjust=False).mean()
  data['ATR'] = data['High'] - data['Low']

  # Добавление Bollinger Bands
  indicator_bb = BollingerBands(close=data['Close'], window=20, window_dev=2)
  data['bb_high'] = indicator_bb.bollinger_hband()
  data['bb_low'] = indicator_bb.bollinger_lband()

  # Добавление Stochastic Oscillator
  indicator_so = StochasticOscillator(high=data['High'], low=data['Low'], close=data['Close'], window=14, smooth_window=3)
  data['so'] = indicator_so.stoch()

  # Формирование признака предсказания для стратегии, основанной на использовании GAP,
  # как отношения цены открытия завтра к цене закрытия сегодня
  data["Predict"] = data['Open'].shift(-1) / data["Close"]

  # Удаляем строки с NaN-значениями
  data.dropna(inplace=True)

  # Сохраняем реальные цены закрытия
  Close_prices = data['Close'].values

  # Нормализация данных только к выбранным колонкам
  scaler = MinMaxScaler(feature_range=(0, 1))
  #scaler = StandardScaler()
  #scaler = RobustScaler()
  scaled_data = scaler.fit_transform(data[columns_to_scale])

  # Преобразуем результат обратно в DataFrame
  scaled_df = pd.DataFrame(scaled_data, columns=columns_to_scale)

  # Добавляем колонки, которые не нужно было нормализовать
  scaled_df[columns_not_to_scale] = data[columns_not_to_scale].reset_index(drop=True)

  # Сохраняем колонку 'Predict' для целевой переменной
  y_data = scaled_df['Predict'].values

  # Удаление колонки 'Predict' и преобразование DataFrame в массив NumPy для удобства работы
  X_data = scaled_df.drop(columns=['Predict']).values

  # Создание временных окон
  X, y = [], []
  for i in range(window_size, len(scaled_df)):
      X.append(X_data[i - window_size:i, :]) # Все колонки, кроме 'Predict'
      y.append(y_data[i]) # Целевая переменная колонка 'Predict'
  X, y = np.array(X), np.array(y)

  # Разделение на обучающую и тестовую выборки
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

  # Разделение Close_prices в том же отношении, что и X, y
  # Используем индексы, полученные при разбиении X и y
  split_index = len(X_train)
  Close_train = Close_prices[window_size:split_index + window_size]
  Close_test = Close_prices[split_index + window_size:]

  return X_train, X_test, y_train, y_test, Close_train, Close_test, scaler

In [115]:
def create_lstm_model(input_shape):
  model = Sequential(name='lstm_model')
  model.add(LSTM(50, return_sequences=True, input_shape=input_shape))
  model.add(Dropout(0.2))
  model.add(LSTM(50, return_sequences=False))
  model.add(Dropout(0.2))
  model.add(Dense(25))
  model.add(Dense(1))  # Прогнозирование цены
  model.compile(optimizer='adam', loss='mean_squared_error')
  return model

In [116]:
def create_optimized_lstm_model(input_shape):
    model = Sequential(name='optimized_lstm_model')
    model.add(LSTM(50, return_sequences=True, input_shape=input_shape, kernel_regularizer=l2(0.01)))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(LSTM(50, return_sequences=False, kernel_regularizer=l2(0.01)))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(25, kernel_regularizer=l2(0.01)))
    model.add(BatchNormalization())
    model.add(Dense(1))  # Прогнозирование цены
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [117]:
def create_cnn_lstm_model(input_shape):
    model = Sequential(name='cnn_lstm_model')
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2))
    model.add(LSTM(50, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(50, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(25))
    model.add(Dense(1))  # Прогнозирование цены
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [118]:
def create_optimized_cnn_lstm_model(input_shape):
    model = Sequential(name='optimized_cnn_lstm_model')
    # Слой Conv1D с регуляризацией L2
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape, kernel_regularizer=l2(0.01)))
    model.add(MaxPooling1D(pool_size=2))
    # Пакетная нормализация после Conv1D
    model.add(BatchNormalization())
    # Первый LSTM слой с регуляризацией L2 и пакетной нормализацией
    model.add(LSTM(50, return_sequences=True, kernel_regularizer=l2(0.01)))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    # Второй LSTM слой с регуляризацией L2 и пакетной нормализацией
    model.add(LSTM(50, return_sequences=False, kernel_regularizer=l2(0.01)))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    # Полносвязный слой с регуляризацией L2 и пакетной нормализацией
    model.add(Dense(25, kernel_regularizer=l2(0.01)))
    model.add(BatchNormalization())
    # Выходной слой
    model.add(Dense(1))  # Прогнозирование цены
    # Компиляция модели
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [119]:
def create_gru_model(input_shape):
    model = Sequential(name='gru_model')
    model.add(GRU(50, return_sequences=True, input_shape=input_shape, kernel_regularizer=l2(0.01)))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(GRU(50, return_sequences=False, kernel_regularizer=l2(0.01)))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(25, kernel_regularizer=l2(0.01)))
    model.add(BatchNormalization())
    model.add(Dense(1))  # Прогнозирование цены
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [120]:
def train_model(model, X_train, y_train, epochs=50, batch_size=32):
  history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.1, verbose=1)
  return history

In [121]:
def test_model(model, X_test, y_test):
    # Получаем прогнозы
    predictions = model.predict(X_test)
    return predictions, y_test

In [122]:
def calculate_metrics(predictions, y_test):
  # метрика: точность направления движения цены
  pred_sign = np.sign(predictions -1)
  test_sign = np.sign(y_test - 1)
  direction_accuracy = accuracy_score(test_sign, pred_sign)
  rmse = np.mean((predictions - y_test) ** 2) ** 0.5
  mae = np.mean(np.abs(predictions - y_test))
  average = y_test.mean()
  rmse2avg = rmse / average
  mae2avg = mae / average
  return direction_accuracy, rmse, mae, rmse2avg, mae2avg

In [123]:
# Загрузка данных
#file_path = 'stock_data.csv'  # Замените на путь к вашему файлу
#data = load_data_from_file(file_path)
data = load_data_from_yf("AAPL")

[*********************100%***********************]  1 of 1 completed


In [124]:
# Предобработка данных
X_train, X_test, y_train, y_test, Close_train, Close_test, scaler = preprocess_data(data)

In [125]:
# Создание и обучение модели
#model = create_lstm_model((X_train.shape[1], X_train.shape[2]))
#model = create_optimized_lstm_model((X_train.shape[1], X_train.shape[2]))
#model = create_cnn_lstm_model((X_train.shape[1], X_train.shape[2]))
model = create_optimized_cnn_lstm_model((X_train.shape[1], X_train.shape[2]))
history = train_model(model, X_train, y_train, 10)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 14s 27ms/step - loss: 2.4822 - val_loss: 0.7037
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 9s 21ms/step - loss: 0.5767 - val_loss: 0.3149
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 6s 24ms/step - loss: 0.2737 - val_loss: 0.1685
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 14s 40ms/step - loss: 0.1484 - val_loss: 0.0928
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - loss: 0.0824 - val_loss: 0.0516
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - loss: 0.0457 - val_loss: 0.0288
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 6s 25ms/step - loss: 0.0256 - val_loss: 0.0163
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - loss: 0.0152 - val_loss: 0.0100
Epoch 9/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 6s 25ms/step - loss: 0.0098 - val_loss: 0.0068
Epoch 10/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - loss: 0.0066 - val_loss: 0.0045


In [126]:
# Тестирование модели
predictions, y_test = test_model(model, X_test, y_test)

# Расчет метрик
direction_accuracy, rmse, mae, rmse2avg, mae2avg  = calculate_metrics(predictions, y_test)
print(f"Точность направления движения цены: {direction_accuracy:.4f}")
print(f"Средняя абсолютная ошибка цены (MAE): {mae:.4f}")
print(f"Средняя квадратичная ошибка цены (RMSE): {rmse:.4f}")
print(f"Относительная средняя абсолютная ошибка цены (MAE): {mae2avg:.4f}")
print(f"Относительная средняя квадратичная ошибка цены (RMSE): {rmse2avg:.4f}")

70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step
Точность направления движения цены: 0.4817
Средняя абсолютная ошибка цены (MAE): 0.0091
Средняя квадратичная ошибка цены (RMSE): 0.0132
Относительная средняя абсолютная ошибка цены (MAE): 0.0091
Относительная средняя квадратичная ошибка цены (RMSE): 0.0132


In [127]:
count_less_than_1 = np.sum(predictions < 1)
count_great_than_1 = np.sum(predictions > 1)
print(f"Количество элементов меньше 1: {count_less_than_1}")
print(f"Количество элементов больше 1: {count_great_than_1}")

Количество элементов меньше 1: 1280
Количество элементов больше 1: 937


In [128]:
# Сохранение метрик в файл
metrics = {
    'Модель': model.name,
    'Точность направления': direction_accuracy,
    'Средняя абсолютная ошибка': mae,
    'Средняя квадратичная ошибка': rmse,
    'Относительная средняя абсолютная ошибка': mae2avg,
    'Относительная средняя квадратичная ошибка': rmse2avg
}
metrics_df = pd.DataFrame([metrics])
metrics_df.to_csv('metrics.csv', index=False)

In [129]:
# Создание дашборда с использованием Streamlit
def create_dashboard(data, predictions, y_test):
    st.title('Дашборд торговой стратегии')
    st.write("### График прогнозов и реальных цен")
    plt.figure(figsize=(10, 6))
    plt.plot(y_test, label='Реальные цены')
    plt.plot(predictions, label='Прогнозы')
    plt.legend()
    st.pyplot(plt)

    st.write("### Метрики модели")
    metrics_df = pd.read_csv('metrics.csv')
    st.table(metrics_df)

In [130]:
create_dashboard(data, predictions, y_test)

2025-03-02 20:27:45.272 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-02 20:27:45.274 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-02 20:27:45.275 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-02 20:27:45.276 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-02 20:27:45.277 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-02 20:27:45.278 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-02 20:27:45.294 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-02 20:27:45.739 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [131]:
# Создание стратегии для backtrader
class SimulationStrategy(bt.Strategy):
    def __init__(self):
        self.prediction_index = 0  # Индекс для отслеживания текущего предсказания

    def next(self):
        if self.prediction_index < len(predictions):
            prediction = predictions[self.prediction_index]
            if prediction > 1:  # Если предсказание роста цены
                if not self.position:
                    self.buy()  # Открываем длинную позицию
            elif prediction < 1:  # Если предсказание падения цены
                if self.position:
                    self.sell()  # Закрываем позицию
            self.prediction_index += 1

In [140]:
# Создание экземпляра Cerebro
cerebro = bt.Cerebro()

# Добавление стратегии
cerebro.addstrategy(SimulationStrategy)

# Подготовка тестовых данных для backtrader
test_data = data.iloc[-len(X_test):]  # Выбираем тестовую часть данных
test_data = bt.feeds.PandasData(dataname=test_data)

# Добавление данных
cerebro.adddata(test_data)

# Добавление анализаторов
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="ta")
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name="sharpe")
cerebro.addanalyzer(bt.analyzers.DrawDown, _name="drawdown")
cerebro.addanalyzer(bt.analyzers.TimeReturn, _name='timereturn')

# Установка начального капитала
cerebro.broker.set_cash(10000.0)

# Запуск симуляции с анализаторами
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
analysis = cerebro.run()
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 10000.00
Final Portfolio Value: 10075.14


In [145]:
# Вывод дополнительных характеристик
def print_trade_analysis(analyzer):
    print("\n--- Trade Analysis ---")
    print(f"Total Trades: {analyzer.total.total}")
    print(f"Total Closed Trades: {analyzer.total.closed}")
    print(f"Total Open Trades: {analyzer.total.open}")
    print(f"Total Won: {analyzer.won.total}")
    print(f"Total Lost: {analyzer.lost.total}")
    print(f"Win Rate: {analyzer.won.total / analyzer.total.closed * 100:.2f}%")
    print(f"Average Win: {analyzer.won.pnl.average:.2f}")
    print(f"Average Loss: {analyzer.lost.pnl.average:.2f}")
    #print(f"Profit Factor: {analyzer.pnl.net.profit_factor:.2f}")

def print_sharpe_ratio(analyzer):
    print("\n--- Sharpe Ratio ---")
    print(f"Sharpe Ratio: {analyzer.get('sharperatio', 'N/A'):.2f}")

def print_drawdown(analyzer):
    print("\n--- Drawdown ---")
    print(f"Max Drawdown: {analyzer.max.drawdown:.2f}%")
    print(f"Max Money Drawdown: {analyzer.max.moneydown:.2f}")

# Вывод результатов анализа
print_trade_analysis(analysis[0].analyzers.ta.get_analysis())
print_sharpe_ratio(analysis[0].analyzers.sharpe.get_analysis())
print_drawdown(analysis[0].analyzers.drawdown.get_analysis())
capital_history = analysis[0].analyzers.timereturn.get_analysis().items()

# Визуализация результатов
cerebro.plot()


--- Trade Analysis ---
Total Trades: 51
Total Closed Trades: 50
Total Open Trades: 1
Total Won: 30
Total Lost: 20
Win Rate: 60.00%
Average Win: 5.01
Average Loss: -4.14

--- Sharpe Ratio ---
Sharpe Ratio: -6.87

--- Drawdown ---
Max Drawdown: 0.36%
Max Money Drawdown: 36.34


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

[[<Figure size 640x480 with 4 Axes>]]

In [146]:
# Добавление графика капитала в дашборд
def create_dashboard2(data, predictions, y_test, capital_history):
    st.title('Дашборд торговой стратегии')
    st.write("### График прогнозов и реальных цен")
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(y_test, label='Реальные цены')
    ax.plot(predictions, label='Прогнозы')
    ax.legend()
    st.pyplot(fig)

    st.write("### График изменения капитала")
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(capital_history, label='Капитал')
    ax.set_xlabel('Время')
    ax.set_ylabel('Капитал')
    ax.legend()
    st.pyplot(fig)

    st.write("### Метрики модели")
    metrics_df = pd.read_csv('metrics.csv')
    st.table(metrics_df)

In [147]:
create_dashboard2(data, predictions, y_test, capital_history)

2025-03-02 20:46:18.048 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-02 20:46:18.050 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-02 20:46:18.051 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-02 20:46:18.052 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-02 20:46:18.052 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-02 20:46:18.053 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-02 20:46:18.073 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-02 20:46:18.527 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

ВЫВОДЫ:
1. Модель плохо обучается по предсказыванию целевой величины и поэтому всегда предсказывает только повышение
2. Торговая стратегия правктически не генерирует прибыль. Предположительно это из-за того, что нормировать нужно каждое временное окно отдельно, а не весь ряд целиком как единое целое.